In [ ]:
# Install neqsim if not already installed
try:
    import neqsim
except ImportError:
    !pip install neqsim

# Import required modules
import jpype
import jpype.imports
import numpy as np
import matplotlib.pyplot as plt

# Start JVM if not already running
if not jpype.isJVMStarted():
    from neqsim import jNeqSim

# Import Java classes for direct access
from neqsim.thermo.system import SystemSrkEos

# Direct Java imports for fluid mechanics classes
TwoPhasePipeFlowSystem = jpype.JClass('neqsim.fluidmechanics.flowsystem.twophaseflowsystem.twophasepipeflowsystem.TwoPhasePipeFlowSystem')
FlowPattern = jpype.JClass('neqsim.fluidmechanics.flownode.FlowPattern')
FlowPatternModel = jpype.JClass('neqsim.fluidmechanics.flownode.FlowPatternModel')
PipeData = jpype.JClass('neqsim.fluidmechanics.geometrydefinitions.pipe.PipeData')

print("NeqSim loaded successfully!")

## 1. Create a Two-Phase Fluid System

We'll create a natural gas system with water, typical of wet gas pipelines.

In [ ]:
# Create a two-phase system (natural gas with water)
# Temperature: 20°C (293.15 K), Pressure: 50 bar
fluid = SystemSrkEos(293.15, 50.0)

# Add gas phase components
fluid.addComponent("methane", 0.85, 0)  # 85 mol% methane in gas
fluid.addComponent("ethane", 0.08, 0)   # 8 mol% ethane
fluid.addComponent("propane", 0.04, 0)  # 4 mol% propane
fluid.addComponent("CO2", 0.02, 0)      # 2 mol% CO2
fluid.addComponent("water", 0.01, 0)    # 1 mol% water vapor

# Add liquid phase (water)
fluid.addComponent("water", 0.1, 1)     # Liquid water

# Initialize the database and mixing rule
fluid.createDatabase(True)
fluid.setMixingRule(2)  # Classic mixing rule

# Print initial conditions
print(f"Temperature: {fluid.getTemperature()} K ({fluid.getTemperature() - 273.15:.1f} °C)")
print(f"Pressure: {fluid.getPressure()} bar")
print(f"Number of components: {fluid.getNumberOfComponents()}")

## 2. Build the Two-Phase Pipe Flow System

Using the builder pattern for clean, readable configuration.

In [ ]:
# Create the pipe system using the builder pattern
pipe = TwoPhasePipeFlowSystem.builder() \
    .withFluid(fluid) \
    .withDiameter(0.15, "m") \
    .withLength(500, "m") \
    .withNodes(50) \
    .withFlowPattern(FlowPattern.STRATIFIED) \
    .build()

print(f"Pipe created with:")
print(f"  - Diameter: 0.15 m")
print(f"  - Length: 500 m")
print(f"  - Nodes: {pipe.getTotalNumberOfNodes()}")
print(f"  - Initial flow pattern: Stratified")

## 3. Temperature and Pressure Profiles

Visualize how temperature and pressure change along the pipe.

In [ ]:
# Get profiles from the pipe system
temperature_profile = np.array(pipe.getTemperatureProfile())
pressure_profile = np.array(pipe.getPressureProfile())
length_profile = np.array(pipe.getLengthProfile())

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Temperature profile
ax1 = axes[0]
ax1.plot(length_profile, temperature_profile - 273.15, 'r-', linewidth=2)
ax1.set_xlabel('Position along pipe (m)', fontsize=12)
ax1.set_ylabel('Temperature (°C)', fontsize=12)
ax1.set_title('Temperature Profile', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.fill_between(length_profile, temperature_profile - 273.15, alpha=0.2, color='red')

# Pressure profile
ax2 = axes[1]
ax2.plot(length_profile, pressure_profile, 'b-', linewidth=2)
ax2.set_xlabel('Position along pipe (m)', fontsize=12)
ax2.set_ylabel('Pressure (bar)', fontsize=12)
ax2.set_title('Pressure Profile', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.fill_between(length_profile, pressure_profile, alpha=0.2, color='blue')

plt.tight_layout()
plt.show()

print(f"\nInlet Temperature: {temperature_profile[0] - 273.15:.2f} °C")
print(f"Outlet Temperature: {temperature_profile[-1] - 273.15:.2f} °C")
print(f"Temperature Change: {temperature_profile[-1] - temperature_profile[0]:.2f} K")
print(f"\nInlet Pressure: {pressure_profile[0]:.2f} bar")
print(f"Outlet Pressure: {pressure_profile[-1]:.2f} bar")
print(f"Pressure Drop: {pressure_profile[0] - pressure_profile[-1]:.4f} bar")

## 4. Velocity and Phase Fraction Profiles

In [ ]:
# Get velocity profiles
gas_velocity = np.array(pipe.getVelocityProfile(0))
liquid_velocity = np.array(pipe.getVelocityProfile(1))
liquid_holdup = np.array(pipe.getLiquidHoldupProfile())

# Create subplots
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Velocity profiles
ax1 = axes[0]
ax1.plot(length_profile, gas_velocity, 'g-', linewidth=2, label='Gas')
ax1.plot(length_profile, liquid_velocity, 'b-', linewidth=2, label='Liquid')
ax1.set_xlabel('Position along pipe (m)', fontsize=12)
ax1.set_ylabel('Velocity (m/s)', fontsize=12)
ax1.set_title('Phase Velocity Profiles', fontsize=14)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Liquid holdup
ax2 = axes[1]
ax2.fill_between(length_profile, liquid_holdup, alpha=0.6, color='blue', label='Liquid')
ax2.fill_between(length_profile, liquid_holdup, 1.0, alpha=0.6, color='green', label='Gas')
ax2.set_xlabel('Position along pipe (m)', fontsize=12)
ax2.set_ylabel('Phase Fraction (-)', fontsize=12)
ax2.set_title('Liquid Holdup Profile', fontsize=14)
ax2.legend(loc='best')
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"Average Gas Velocity: {np.mean(gas_velocity):.2f} m/s")
print(f"Average Liquid Velocity: {np.mean(liquid_velocity):.2f} m/s")
print(f"Slip Ratio (Gas/Liquid): {np.mean(gas_velocity)/np.mean(liquid_velocity):.2f}")

## 5. Heat Transfer Coefficients

Analyze interphase heat transfer coefficients and dimensionless numbers.

In [ ]:
# Get heat transfer coefficients
liquid_htc = np.array(pipe.getLiquidHeatTransferCoefficientProfile())
gas_htc = np.array(pipe.getGasHeatTransferCoefficientProfile())
overall_htc = np.array(pipe.getOverallInterphaseHeatTransferCoefficientProfile())

# Get dimensionless numbers
reynolds_gas = np.array(pipe.getReynoldsNumberProfile(0))
reynolds_liquid = np.array(pipe.getReynoldsNumberProfile(1))
prandtl_gas = np.array(pipe.getPrandtlNumberProfile(0))
nusselt_gas = np.array(pipe.getNusseltNumberProfile(0))

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Heat transfer coefficients
ax1 = axes[0, 0]
ax1.semilogy(length_profile, liquid_htc, 'b-', linewidth=2, label='Liquid HTC')
ax1.semilogy(length_profile, gas_htc, 'g-', linewidth=2, label='Gas HTC')
ax1.semilogy(length_profile, overall_htc, 'r--', linewidth=2, label='Overall HTC')
ax1.set_xlabel('Position (m)', fontsize=12)
ax1.set_ylabel('Heat Transfer Coefficient (W/m²·K)', fontsize=12)
ax1.set_title('Interphase Heat Transfer Coefficients', fontsize=14)
ax1.legend(loc='best')
ax1.grid(True, alpha=0.3)

# Reynolds numbers
ax2 = axes[0, 1]
ax2.semilogy(length_profile, reynolds_gas, 'g-', linewidth=2, label='Gas Re')
ax2.semilogy(length_profile, reynolds_liquid, 'b-', linewidth=2, label='Liquid Re')
ax2.axhline(y=2300, color='k', linestyle='--', label='Transition (2300)')
ax2.set_xlabel('Position (m)', fontsize=12)
ax2.set_ylabel('Reynolds Number (-)', fontsize=12)
ax2.set_title('Reynolds Number Profiles', fontsize=14)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

# Prandtl and Nusselt numbers
ax3 = axes[1, 0]
ax3.plot(length_profile, prandtl_gas, 'm-', linewidth=2)
ax3.set_xlabel('Position (m)', fontsize=12)
ax3.set_ylabel('Prandtl Number (-)', fontsize=12)
ax3.set_title('Gas Prandtl Number Profile', fontsize=14)
ax3.grid(True, alpha=0.3)

ax4 = axes[1, 1]
ax4.plot(length_profile, nusselt_gas, 'c-', linewidth=2)
ax4.set_xlabel('Position (m)', fontsize=12)
ax4.set_ylabel('Nusselt Number (-)', fontsize=12)
ax4.set_title('Gas Nusselt Number Profile', fontsize=14)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nHeat Transfer Summary:")
print(f"  Average Liquid HTC: {np.mean(liquid_htc):.1f} W/(m²·K)")
print(f"  Average Gas HTC: {np.mean(gas_htc):.1f} W/(m²·K)")
print(f"  Average Overall HTC: {np.mean(overall_htc):.1f} W/(m²·K)")
print(f"\nDimensionless Numbers (Gas phase):")
print(f"  Reynolds: {np.mean(reynolds_gas):.0f} ({'Turbulent' if np.mean(reynolds_gas) > 2300 else 'Laminar'})")
print(f"  Prandtl: {np.mean(prandtl_gas):.3f}")
print(f"  Nusselt: {np.mean(nusselt_gas):.1f}")

## 6. Flow Pattern Detection

Enable automatic flow pattern detection and visualize the results.

In [ ]:
# Enable automatic flow pattern detection using Taitel-Dukler model
pipe.setFlowPatternModel(FlowPatternModel.TAITEL_DUKLER)
pipe.detectFlowPatterns()

# Get flow pattern profile
flow_patterns = pipe.getFlowPatternNameProfile()
flow_pattern_names = [str(p) for p in flow_patterns]

# Count occurrences of each pattern
unique_patterns = list(set(flow_pattern_names))
pattern_counts = {p: flow_pattern_names.count(p) for p in unique_patterns}

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Flow pattern map
ax1 = axes[0]
pattern_colors = {
    'STRATIFIED': 'blue',
    'STRATIFIED_WAVY': 'lightblue',
    'ANNULAR': 'green',
    'SLUG': 'orange',
    'BUBBLE': 'purple',
    'DROPLET': 'red',
    'CHURN': 'brown',
    'DISPERSED_BUBBLE': 'pink'
}

for i, pattern in enumerate(flow_pattern_names):
    color = pattern_colors.get(pattern, 'gray')
    ax1.axvspan(length_profile[i] - 5, length_profile[i] + 5 if i < len(length_profile)-1 else length_profile[i], 
                color=color, alpha=0.5)

# Add legend
from matplotlib.patches import Patch
legend_elements = [Patch(facecolor=color, alpha=0.5, label=pattern) 
                   for pattern, color in pattern_colors.items() if pattern in unique_patterns]
ax1.legend(handles=legend_elements, loc='upper right')
ax1.set_xlabel('Position along pipe (m)', fontsize=12)
ax1.set_title('Flow Pattern Map Along Pipe', fontsize=14)
ax1.set_yticks([])

# Pie chart of flow pattern distribution
ax2 = axes[1]
colors = [pattern_colors.get(p, 'gray') for p in pattern_counts.keys()]
ax2.pie(pattern_counts.values(), labels=pattern_counts.keys(), colors=colors,
        autopct='%1.1f%%', startangle=90)
ax2.set_title('Flow Pattern Distribution', fontsize=14)

plt.tight_layout()
plt.show()

print(f"\nFlow Pattern Statistics:")
print(f"  Number of transitions: {pipe.getFlowPatternTransitionCount()}")
for pattern, count in pattern_counts.items():
    print(f"  {pattern}: {count} nodes ({100*count/len(flow_pattern_names):.1f}%)")

## 7. Pressure Drop Analysis

Breakdown of pressure drop components: frictional, gravitational, and acceleration.

In [ ]:
# Get pressure drop breakdown
total_dp = pipe.getTotalPressureDrop()
frictional_dp = pipe.getFrictionalPressureDrop()
gravitational_dp = pipe.getGravitationalPressureDrop()
acceleration_dp = pipe.getAccelerationPressureDrop()

# Get Lockhart-Martinelli pressure gradient
lm_gradient = np.array(pipe.getLockhartMartinelliPressureGradientProfile())
actual_gradient = np.array(pipe.getPressureGradientProfile())

# Create visualizations
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Bar chart of pressure drop components
ax1 = axes[0]
components = ['Total', 'Frictional', 'Gravitational', 'Acceleration']
values = [total_dp, frictional_dp, gravitational_dp, acceleration_dp]
colors = ['darkblue', 'red', 'green', 'orange']
bars = ax1.bar(components, values, color=colors, alpha=0.7)
ax1.set_ylabel('Pressure Drop (bar)', fontsize=12)
ax1.set_title('Pressure Drop Breakdown', fontsize=14)
ax1.grid(True, alpha=0.3, axis='y')

# Add value labels on bars
for bar, val in zip(bars, values):
    height = bar.get_height()
    ax1.text(bar.get_x() + bar.get_width()/2., height,
             f'{val:.4f}',
             ha='center', va='bottom', fontsize=10)

# Compare pressure gradients
ax2 = axes[1]
ax2.plot(length_profile, actual_gradient, 'b-', linewidth=2, label='Simulated')
ax2.plot(length_profile, lm_gradient, 'r--', linewidth=2, label='Lockhart-Martinelli')
ax2.set_xlabel('Position (m)', fontsize=12)
ax2.set_ylabel('Pressure Gradient (Pa/m)', fontsize=12)
ax2.set_title('Pressure Gradient Comparison', fontsize=14)
ax2.legend(loc='best')
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print pressure drop breakdown
print(pipe.getPressureDropBreakdown())

## 8. Mass Transfer Analysis

In [ ]:
# Get mass transfer related profiles
interfacial_area = np.array(pipe.getSpecificInterfacialAreaProfile())
gas_quality = np.array(pipe.getGasQualityProfile())

# Diffusivity for mass transfer coefficient calculation (typical value for gas diffusion)
diffusivity_gas = 2.0e-5  # m²/s
diffusivity_liquid = 1.0e-9  # m²/s

liquid_mtc = np.array(pipe.getLiquidMassTransferCoefficientProfile(diffusivity_liquid))
gas_mtc = np.array(pipe.getGasMassTransferCoefficientProfile(diffusivity_gas))

# Create subplots
fig, axes = plt.subplots(2, 2, figsize=(14, 10))

# Interfacial area
ax1 = axes[0, 0]
ax1.plot(length_profile, interfacial_area, 'purple', linewidth=2)
ax1.set_xlabel('Position (m)', fontsize=12)
ax1.set_ylabel('Specific Interfacial Area (1/m)', fontsize=12)
ax1.set_title('Specific Interfacial Area Profile', fontsize=14)
ax1.grid(True, alpha=0.3)

# Gas quality
ax2 = axes[0, 1]
ax2.plot(length_profile, gas_quality * 100, 'orange', linewidth=2)
ax2.set_xlabel('Position (m)', fontsize=12)
ax2.set_ylabel('Gas Quality (%)', fontsize=12)
ax2.set_title('Gas Quality Profile (Mass Fraction)', fontsize=14)
ax2.grid(True, alpha=0.3)

# Mass transfer coefficients
ax3 = axes[1, 0]
ax3.semilogy(length_profile, liquid_mtc, 'b-', linewidth=2, label='Liquid k_L')
ax3.semilogy(length_profile, gas_mtc, 'g-', linewidth=2, label='Gas k_G')
ax3.set_xlabel('Position (m)', fontsize=12)
ax3.set_ylabel('Mass Transfer Coefficient (m/s)', fontsize=12)
ax3.set_title('Mass Transfer Coefficients', fontsize=14)
ax3.legend(loc='best')
ax3.grid(True, alpha=0.3)

# Volumetric mass transfer coefficient
ax4 = axes[1, 1]
kLa = liquid_mtc * interfacial_area
ax4.plot(length_profile, kLa, 'teal', linewidth=2)
ax4.set_xlabel('Position (m)', fontsize=12)
ax4.set_ylabel('k_L·a (1/s)', fontsize=12)
ax4.set_title('Volumetric Mass Transfer Coefficient', fontsize=14)
ax4.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nMass Transfer Summary:")
print(f"  Average Interfacial Area: {np.mean(interfacial_area):.2f} 1/m")
print(f"  Average Liquid k_L: {np.mean(liquid_mtc):.2e} m/s")
print(f"  Average Gas k_G: {np.mean(gas_mtc):.2e} m/s")
print(f"  Average k_L·a: {np.mean(kLa):.2e} 1/s")

## 9. Energy and Mass Balance Verification

In [ ]:
# Get balance errors
mass_error = pipe.getMassBalanceError()
energy_error = pipe.getEnergyBalanceError()

# Get cumulative energy loss
cumulative_energy_loss = np.array(pipe.getCumulativeEnergyLossProfile())
total_mass_flow = np.array(pipe.getTotalMassFlowRateProfile())

# Create visualizations
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Balance errors
ax1 = axes[0]
categories = ['Mass Balance', 'Energy Balance']
errors = [mass_error * 100, energy_error * 100]
colors = ['green' if e < 5 else 'orange' if e < 10 else 'red' for e in [abs(mass_error)*100, abs(energy_error)*100]]
bars = ax1.bar(categories, errors, color=colors, alpha=0.7)
ax1.axhline(y=5, color='orange', linestyle='--', label='5% threshold')
ax1.axhline(y=10, color='red', linestyle='--', label='10% threshold')
ax1.set_ylabel('Error (%)', fontsize=12)
ax1.set_title('Conservation Balance Errors', fontsize=14)
ax1.legend(loc='upper right')
ax1.grid(True, alpha=0.3, axis='y')

# Cumulative energy loss
ax2 = axes[1]
ax2.plot(length_profile, cumulative_energy_loss / 1000, 'red', linewidth=2)
ax2.fill_between(length_profile, cumulative_energy_loss / 1000, alpha=0.3, color='red')
ax2.set_xlabel('Position (m)', fontsize=12)
ax2.set_ylabel('Cumulative Energy Loss (kW)', fontsize=12)
ax2.set_title('Cumulative Energy Loss Profile', fontsize=14)
ax2.grid(True, alpha=0.3)

# Total mass flow
ax3 = axes[2]
ax3.plot(length_profile, total_mass_flow, 'navy', linewidth=2)
ax3.set_xlabel('Position (m)', fontsize=12)
ax3.set_ylabel('Total Mass Flow Rate (kg/s)', fontsize=12)
ax3.set_title('Total Mass Flow Rate Profile', fontsize=14)
ax3.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nConservation Balance Summary:")
print(f"  Mass Balance Error: {mass_error * 100:.4f}% {'✓' if abs(mass_error) < 0.05 else '✗'}")
print(f"  Energy Balance Error: {energy_error * 100:.4f}% {'✓' if abs(energy_error) < 0.1 else '✗'}")
print(f"\n  Total Energy Loss: {cumulative_energy_loss[-1]/1000:.2f} kW")
print(f"  Inlet Mass Flow: {total_mass_flow[0]:.4f} kg/s")
print(f"  Outlet Mass Flow: {total_mass_flow[-1]:.4f} kg/s")

## 10. Summary Dashboard

In [ ]:
# Create a comprehensive summary
print("=" * 60)
print("TWO-PHASE PIPE FLOW SIMULATION SUMMARY")
print("=" * 60)

print("\n📐 GEOMETRY:")
print(f"   Pipe Diameter: 0.15 m")
print(f"   Pipe Length: 500 m")
print(f"   Number of Nodes: {pipe.getTotalNumberOfNodes()}")

print("\n🌡️ THERMODYNAMIC CONDITIONS:")
print(f"   Inlet Temperature: {temperature_profile[0] - 273.15:.2f} °C")
print(f"   Outlet Temperature: {temperature_profile[-1] - 273.15:.2f} °C")
print(f"   Inlet Pressure: {pressure_profile[0]:.2f} bar")
print(f"   Outlet Pressure: {pressure_profile[-1]:.2f} bar")

print("\n🔄 FLOW CONDITIONS:")
print(f"   Average Gas Velocity: {np.mean(gas_velocity):.2f} m/s")
print(f"   Average Liquid Velocity: {np.mean(liquid_velocity):.2f} m/s")
print(f"   Average Liquid Holdup: {np.mean(liquid_holdup):.4f}")
print(f"   Predominant Flow Pattern: {max(pattern_counts, key=pattern_counts.get)}")

print("\n🔥 HEAT TRANSFER:")
print(f"   Average Interphase HTC: {np.mean(overall_htc):.1f} W/(m²·K)")
print(f"   Total Heat Loss: {cumulative_energy_loss[-1]/1000:.2f} kW")

print("\n💧 MASS TRANSFER:")
print(f"   Average Interfacial Area: {np.mean(interfacial_area):.2f} 1/m")
print(f"   Average k_L·a: {np.mean(kLa):.2e} 1/s")

print("\n📊 PRESSURE DROP:")
print(f"   Total: {total_dp:.4f} bar")
print(f"   Frictional: {frictional_dp:.4f} bar ({100*frictional_dp/total_dp:.1f}%)")
print(f"   Gravitational: {gravitational_dp:.4f} bar ({100*gravitational_dp/total_dp:.1f}%)")
print(f"   Acceleration: {acceleration_dp:.4f} bar ({100*acceleration_dp/total_dp:.1f}%)")

print("\n✅ CONSERVATION BALANCE:")
print(f"   Mass Balance Error: {mass_error * 100:.4f}%")
print(f"   Energy Balance Error: {energy_error * 100:.4f}%")

print("\n" + "=" * 60)

## References

1. Solbraa, E. (2002). "Measurement and Calculation of Two-Phase Flow in Pipes." PhD Thesis, Norwegian University of Science and Technology.

2. Taitel, Y., & Dukler, A.E. (1976). "A model for predicting flow regime transitions in horizontal and near horizontal gas-liquid flow." AIChE Journal, 22(1), 47-55.

3. Lockhart, R.W. and Martinelli, R.C. (1949). "Proposed Correlation of Data for Isothermal Two-Phase, Two-Component Flow in Pipes." Chemical Engineering Progress, 45(1), 39-48.

4. Krishna, R. and Standart, G.L. (1976). "A multicomponent film model incorporating a general matrix method of solution to the Maxwell-Stefan equations." AIChE Journal, 22(2), 383-389.